# 🌩️ Torrent to Google Drive Converter

**Convert any torrent file or magnet link directly to your Google Drive!**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Torrent_to_Google_Drive.ipynb)

## 🚀 Quick Start Guide

This notebook lets you download torrents directly to your Google Drive using Google Colab's free cloud servers!

### ✨ Features:
- ✅ **Free & No Installation** - Runs on Google Colab
- ✅ **Fast Downloads** - Uses Google's high-speed servers
- ✅ **Direct to Drive** - Files saved automatically to your Google Drive
- ✅ **No Storage Needed** - Uses your Google Drive space, not local storage
- ✅ **Multiple Formats** - Supports .torrent files and magnet links
- ✅ **Live Progress** - Real-time download monitoring with progress bars
- ✅ **Direct Links** - Get clickable links to access your files instantly

### 📋 How to Use:

1. **Click the "Open in Colab" badge above** (or open this notebook in Google Colab)
2. **Run each cell in order** (Click the ▶️ play button or press `Shift + Enter`)
   - Cell 1: Installs required libraries (~30 seconds)
   - Cell 2: Initializes torrent engine
   - Cell 3: Connects your Google Drive
   - Cell 4: Upload .torrent file OR Cell 5: Paste magnet link
   - Cell 6: Watch live download progress
   - Cell 7: Get direct links to your downloaded files
3. **Access your files** - Click the generated links to open in Google Drive!

### ⚠️ Important Notes:

- **Python Version**: If Cell 1 fails, go to `Runtime` → `Change runtime type` → Select `Python 3.10`
- **Session Timeout**: Free Colab sessions last ~12 hours. Downloads will stop if disconnected.
- **Storage**: Files use your Google Drive storage quota
- **Legal Use Only**: Only download content you have rights to access

### 🎯 Example Use Cases:

- Download Linux distributions (Ubuntu, Debian, etc.)
- Get open-source software packages
- Download public domain content
- Archive legal torrents to your Drive

### 💡 Pro Tips:

- Upload multiple .torrent files before running Cell 6 to download in parallel
- Keep the browser tab open while downloading
- Check Cell 7 after downloads complete for quick access links

---

**Ready? Start by running Cell 1 below! ⬇️**

## 📦 Step 1: Install Dependencies

In [ ]:
# Install/upgrade required packages
!python -m pip install --upgrade pip setuptools wheel -q

# Try to install libtorrent with Python 3.12 compatibility
import sys
import subprocess

def try_install():
    """Try multiple installation methods for libtorrent"""
    
    # Check if already available
    try:
        import libtorrent
        print("✅ libtorrent already available!")
        return True
    except:
        pass
    
    print("📦 Installing libtorrent...")
    
    # Method 1: Build from source with Python 3.12 support
    print("🔨 Building from source (this may take 2-3 minutes)...")
    try:
        # Install build dependencies
        subprocess.run(['apt-get', 'update', '-qq'], check=True, capture_output=True)
        subprocess.run(['apt-get', 'install', '-y', '-qq', 
                       'build-essential', 'libboost-all-dev', 
                       'libssl-dev', 'python3-dev'], 
                      check=True, capture_output=True)
        
        # Install libtorrent-rasterbar with pip (builds from source)
        result = subprocess.run([sys.executable, '-m', 'pip', 'install', 
                                'libtorrent', '--no-binary', ':all:'],
                               capture_output=True, text=True, timeout=300)
        
        # Test import
        import libtorrent
        print("✅ Successfully built from source!")
        return True
    except Exception as e:
        print(f"⚠️ Source build failed: {str(e)[:100]}")
    
    # Method 2: Try prebuilt package (if available)
    print("📦 Trying prebuilt package...")
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 
                       'libtorrent', '-q'], check=True, capture_output=True)
        import libtorrent
        print("✅ Installed prebuilt package!")
        return True
    except:
        pass
    
    # Method 3: Downgrade Python temporarily (not recommended but works)
    print("⚠️ libtorrent has issues with Python 3.12")
    print("💡 Solution: Runtime > Change runtime type > Python 3.10")
    return False

if try_install():
    print("\n✅ All dependencies ready!")
else:
    print("\n❌ Installation failed")
    print("📝 Please change runtime to Python 3.10:")
    print("   1. Runtime > Change runtime type")
    print("   2. Select Python 3.10")
    print("   3. Save and run this cell again")
    sys.exit(1)

## 🔧 Step 2: Initialize libtorrent Session

In [ ]:
import libtorrent as lt
import time

# Create session
ses = lt.session()
ses.listen_on(6881, 6891)

# Configure settings for better performance
settings = {
    'user_agent': 'libtorrent/2.0',
    'listen_interfaces': '0.0.0.0:6881',
    'enable_outgoing_utp': True,
    'enable_incoming_utp': True,
    'enable_outgoing_tcp': True,
    'enable_incoming_tcp': True,
}
ses.apply_settings(settings)

# Add DHT routers for better peer discovery
ses.add_dht_router("router.utorrent.com", 6881)
ses.add_dht_router("router.bittorrent.com", 6881)
ses.add_dht_router("dht.transmissionbt.com", 6881)
ses.start_dht()

# Store download handles
downloads = []

print("✅ libtorrent session initialized!")
print(f"📡 Listening on ports 6881-6891")

## ☁️ Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create download folder
download_path = "/content/drive/My Drive/Torrents"
os.makedirs(download_path, exist_ok=True)

print(f"✅ Google Drive mounted!")
print(f"📁 Downloads will be saved to: {download_path}")

## 📁 Option A: Upload Torrent File

In [ ]:
from google.colab import files

# Upload torrent file
print("📤 Click 'Choose Files' to upload your .torrent file...")
uploaded = files.upload()

# Add each uploaded torrent
for filename in uploaded.keys():
    print(f"\n📦 Processing: {filename}")
    
    # Create torrent info
    info = lt.torrent_info(filename)
    
    # Set download parameters
    params = {
        'save_path': download_path,
        'ti': info,
        'storage_mode': lt.storage_mode_t.storage_mode_sparse,
    }
    
    # Add torrent to session
    handle = ses.add_torrent(params)
    downloads.append(handle)
    
    print(f"✅ Added: {info.name()}")
    print(f"📊 Size: {info.total_size() / (1024**3):.2f} GB")
    print(f"📄 Files: {info.num_files()}")

print(f"\n🎉 {len(uploaded)} torrent(s) added to queue!")

## 🧲 Option B: Add Magnet Link

In [ ]:
# Paste your magnet link here
magnet_link = input("🧲 Paste magnet link: ")

if magnet_link and magnet_link.startswith('magnet:'):
    params = {
        'save_path': download_path,
        'storage_mode': lt.storage_mode_t.storage_mode_sparse,
    }
    
    handle = lt.add_magnet_uri(ses, magnet_link, params)
    downloads.append(handle)
    
    print("✅ Magnet link added!")
    print("⏳ Fetching metadata...")
else:
    print("❌ Invalid magnet link")

## 📊 Step 4: Monitor Downloads (Live Progress)

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

# State strings
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

# Create progress bars
layout = widgets.Layout(width='auto')
style = {'description_width': 'initial'}
progress_bars = [
    widgets.FloatProgress(
        min=0, max=100,
        description='',
        bar_style='info',
        style=style,
        layout=layout
    ) for _ in downloads
]
labels = [
    widgets.Label(value='') for _ in downloads
]

# Display progress bars
for i in range(len(downloads)):
    display(widgets.VBox([labels[i], progress_bars[i]]))

print("\n🚀 Starting downloads...\n")

# Monitor progress
completed_count = 0
while downloads:
    next_shift = 0
    
    for index, download in enumerate(downloads[:]):
        try:
            # Check if handle is still valid
            if not download.is_valid():
                continue
            
            bar = progress_bars[index + next_shift]
            label = labels[index + next_shift]
            
            # Get status
            s = download.status()
            
            # Check if completed (use status instead of deprecated is_seed)
            if s.is_seeding or s.is_finished:
                # Download complete - save name before removing
                torrent_name = s.name if s.name else "Unknown"
                
                # Update UI
                bar.value = 100
                bar.bar_style = 'success'
                label.value = f"✅ {torrent_name} - COMPLETED!"
                
                # Remove from session and list
                next_shift -= 1
                completed_count += 1
                
                try:
                    ses.remove_torrent(download)
                except:
                    pass  # Already removed
                
                downloads.remove(download)
                
                print(f"\n🎉 Download #{completed_count} complete: {torrent_name}")
                print(f"📁 Saved to: {download_path}")
            else:
                # Still downloading - update progress
                progress = s.progress * 100
                bar.value = progress
                
                # Get download info
                name = s.name if s.name else 'Fetching metadata...'
                dl_speed = s.download_rate / 1000  # KB/s
                ul_speed = s.upload_rate / 1000    # KB/s
                peers = s.num_peers
                state = state_str[s.state] if s.state < len(state_str) else 'unknown'
                
                label.value = (
                    f"📦 {name}\n"
                    f"⬇️ {dl_speed:.1f} KB/s | ⬆️ {ul_speed:.1f} KB/s | "
                    f"👥 {peers} peers | {state} | {progress:.1f}%"
                )
                
                # Color coding
                if progress < 30:
                    bar.bar_style = 'danger'
                elif progress < 70:
                    bar.bar_style = 'warning'
                else:
                    bar.bar_style = 'success'
                    
        except Exception as e:
            # Handle any errors gracefully
            print(f"⚠️ Error monitoring download: {str(e)[:50]}")
            continue
    
    time.sleep(1)

print("\n" + "="*70)
print(f"🎊 All {completed_count} download(s) completed!")
print(f"📂 Check your Google Drive: {download_path}")
print("="*70)

## 📂 Step 5: View Downloaded Files with Direct Links

In [ ]:
import os
from google.colab import auth
from googleapiclient.discovery import build
from IPython.display import HTML, display

# Authenticate and create Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

print("📁 Files in Google Drive/Torrents:\n")
print("="*70)

total_size = 0
file_count = 0
links_html = "<h3>📂 Your Downloaded Files:</h3><ul style='list-style:none; padding:0;'>"

# Get the Torrents folder ID
results = drive_service.files().list(
    q="name='Torrents' and mimeType='application/vnd.google-apps.folder'",
    spaces='drive',
    fields='files(id, name)'
).execute()

torrents_folder_id = None
if results.get('files'):
    torrents_folder_id = results['files'][0]['id']

# List all files in the Torrents folder
if torrents_folder_id:
    page_token = None
    while True:
        results = drive_service.files().list(
            q=f"'{torrents_folder_id}' in parents",
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType, size, webViewLink)',
            pageToken=page_token
        ).execute()
        
        files_list = results.get('files', [])
        
        for file in files_list:
            file_id = file['id']
            file_name = file['name']
            file_size = int(file.get('size', 0))
            mime_type = file['mimeType']
            web_link = file.get('webViewLink', '')
            
            total_size += file_size
            file_count += 1
            
            size_mb = file_size / (1024**2)
            
            # Create clickable link
            if mime_type == 'application/vnd.google-apps.folder':
                icon = "📁"
                size_text = "Folder"
            else:
                icon = "📄"
                size_text = f"{size_mb:.2f} MB"
            
            print(f"{icon} {file_name}")
            print(f"   Size: {size_text}")
            print(f"   🔗 Link: {web_link}")
            print()
            
            # Add to HTML links
            links_html += f"""
            <li style='margin: 15px 0; padding: 10px; background: #f5f5f5; border-radius: 5px;'>
                <strong>{icon} {file_name}</strong><br>
                <span style='color: #666;'>Size: {size_text}</span><br>
                <a href='{web_link}' target='_blank' style='color: #1a73e8; text-decoration: none;'>
                    🔗 Open in Google Drive →
                </a>
            </li>
            """
        
        page_token = results.get('nextPageToken')
        if not page_token:
            break

links_html += "</ul>"

print("="*70)
print(f"📊 Total: {file_count} files, {total_size / (1024**3):.2f} GB")
print(f"☁️ All files are in your Google Drive!")
print("\n📎 Click links below to open files:")

# Display clickable links
display(HTML(links_html))

## 🔄 Optional: Add More Torrents

To add more torrents:
1. Go back to Step 4 (Option A or B)
2. Upload new torrent file or paste magnet link
3. Run Step 5 again to monitor progress

## 💡 Tips:

- **Colab timeout:** Free tier has ~12 hour sessions
- **Storage:** Uses your Google Drive quota
- **Speed:** Depends on seeders and Colab's connection
- **Multiple files:** Upload multiple torrents before monitoring

## 🛑 Stop Downloads

Click Runtime → Interrupt execution to stop all downloads